<font size="6">**S13 T01: Aprenentage Supervisat**</font>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Enunciats" data-toc-modified-id="Enunciats-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Enunciats</a></span></li><li><span><a href="#Importació-i-Preparació-de-Dades" data-toc-modified-id="Importació-i-Preparació-de-Dades-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Importació i Preparació de Dades</a></span></li><li><span><a href="#Selecció-de-variable-objectiu-i-característiques" data-toc-modified-id="Selecció-de-variable-objectiu-i-característiques-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Selecció de variable objectiu i característiques</a></span><ul class="toc-item"><li><span><a href="#Under-Sampling--Near-Miss" data-toc-modified-id="Under-Sampling--Near-Miss-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Under Sampling  Near Miss</a></span></li></ul></li><li><span><a href="#Dividir-les-dades-train-test-split" data-toc-modified-id="Dividir-les-dades-train-test-split-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dividir les dades <code>train-test-split</code></a></span></li><li><span><a href="#Model-3-SVM" data-toc-modified-id="Model-3-SVM-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Model 3 SVM</a></span></li></ul></div>

# Enunciats
**Nivell 1**

- Exercici 1 Crea almenys tres models de classificació diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv. Considera si el vol ha arribat tard o no (ArrDelay > 0).  

- Exercici 2 Compara els models de classificació utilitzant la precisió (accuracy), una matriu de confiança i d’altres mètriques més avançades.  

- Exercici 3 Entrena’ls utilitzant els diferents paràmetres que admeten.

- Exercici 4 Compara el seu rendiment utilitzant l’aproximació traint/test o cross-validation.

**Nivell 2**

- Exercici 5 Realitza algun procés d’enginyeria de variables per millorar-ne la predicció 

**Nivell 3**

- Exercici 6 No utilitzis la variable `DepDelay` a l’hora de fer prediccions

# Importació i Preparació de Dades

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

url='https://github.com/Ortegbel/PractiquesDataScience/blob/master/Exercici5/DelayedFlightsComp.csv.gz?raw=true'
df = pd.read_csv(url,  compression='gzip')


df.head()

,Unnamed: 0,Unnamed: 0.1,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,0,2008,1,3,4,2003.0,1955,2211.0,2225,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,1,2008,1,3,4,754.0,735,1002.0,1000,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2,2008,1,3,4,628.0,620,804.0,750,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,3,4,2008,1,3,4,1829.0,1755,1959.0,1925,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,4,5,2008,1,3,4,1940.0,1915,2121.0,2110,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [2]:
df = df.fillna(0)

In [3]:
df = df.drop(columns=['Unnamed: 0','Unnamed: 0.1' ])
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,N,0,0.0,0.0,0.0,0.0,0.0
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,N,0,0.0,0.0,0.0,0.0,0.0
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,3.0,17.0,0,N,0,0.0,0.0,0.0,0.0,0.0
3,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,...,4.0,10.0,0,N,0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.ArrDelay.unique()

array([ -14.,    2.,   14., ...,  880., 1279.,  991.])

In [5]:
conditions = [(df['ArrDelay']> 0),(df['ArrDelay']<= 0) ]
values = [1, 0]
df['ArrDelay2'] = np.select(conditions, values)

df.ArrDelay2.unique()

array([0, 1])

In [6]:
conditions = [(df['DepDelay']> 0),(df['DepDelay']<= 0) ]
values = [1, 0]
df['DepDelay2'] = np.select(conditions, values)

In [7]:
df['CarrierFzd'] = pd.factorize(df['UniqueCarrier'])[0]
df['OriginFzd'] = pd.factorize(df['Origin'])[0]
df['DestFzd'] = pd.factorize(df['Dest'])[0]

In [8]:
df['TailNumFzd'] = pd.factorize(df['TailNum'])[0]

In [9]:
df.TailNumFzd.unique()

array([   0,    1,    2, ..., 5364, 5365, 5366], dtype=int64)

# Selecció de variable objectiu i característiques

In [10]:
feature_cols = [ 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'DepDelay2','NASDelay',
                'OriginFzd', 'DestFzd','Distance','Cancelled', 'Diverted', 'CarrierDelay','WeatherDelay', 'LateAircraftDelay']

X = df[feature_cols]
X = np.array(X)
y = df.ArrDelay2

## Under Sampling  Near Miss

In [11]:
from collections import Counter
from imblearn.under_sampling import NearMiss

In [12]:
counter = Counter(y)
counter

Counter({0: 213343, 1: 1723415})

In [13]:
undersample = NearMiss(version=1, n_neighbors=3)

In [14]:
X_nm, y_nm = undersample.fit_resample(X, y)
# summarize the new class distribution
counter2 = Counter(y_nm)
print(counter2)

Counter({0: 213343, 1: 213343})


# Dividir les dades `train-test-split`

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_nm,y_nm,test_size=0.02,train_size=0.20,random_state=42)

# Model 3 SVM

In [16]:
from sklearn.preprocessing import RobustScaler 
from sklearn import metrics
from sklearn import svm 
from sklearn.pipeline import make_pipeline


In [17]:
from sklearn import svm 
from sklearn import metrics

pipe = make_pipeline(RobustScaler(),svm.SVC(kernel= 'linear') )


In [18]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('svc', SVC(kernel='linear'))])

In [19]:
y_pred_SVM = pipe.predict(X_test)

In [20]:
from sklearn.metrics import classification_report

In [21]:
print(metrics.confusion_matrix(y_test, y_pred_SVM))
print(classification_report(y_test, y_pred_SVM))

[[2785 1519]
 [1014 3216]]
              precision    recall  f1-score   support

           0       0.73      0.65      0.69      4304
           1       0.68      0.76      0.72      4230

    accuracy                           0.70      8534
   macro avg       0.71      0.70      0.70      8534
weighted avg       0.71      0.70      0.70      8534

